## Instalar dependências

**OBJETIVO:**
Garantir que todas as bibliotecas necessárias ao pré-processamento de texto PT estão disponíveis no runtime.

**Passos:**
  - Instala spaCy (processamento de linguagem natural) e o modelo PT pequeno
    'pt_core_news_sm' (suficiente para tokenização e lematização).
  - Instala utilitários de limpeza de texto:
      * ftfy .......... corrige problemas de codificação Unicode
      * unidecode ..... (opcional) remove acentos (normalização)
      * wordcloud ..... visualização rápida da "nuvem" de termos
      * matplotlib .... gráficos simples (barras/wordcloud)
      * pandas ........ manipulação tabular dos textos
      * scikit-learn .. TF-IDF e vetorização
      * lxml .......... utilitários de parsing (não usados diretamente aqui)

In [ ]:
!pip -q install spacy ftfy unidecode wordcloud matplotlib pandas scikit-learn lxml
!python -m spacy download pt_core_news_sm

**Objetivo:**
Definir um pipeline de pré-processamento textual consistente e seguro para Português, do "texto cru" até tokens lematizados. Transformar textos PT heterogéneos num formato normalizado e informativo, pronto para vetorização (TF-IDF) e visualização.

**Passos:**
  1) Carregamento do spaCy PT com parser/NER desativados:
     - nlp = spacy.load("pt_core_news_sm", disable=["parser","ner"])
     - Motivo: acelerar; aqui só precisamos de tokenização + lemas.

  2) Conjuntos de exclusão (ruído):
     - PT_STOP .......... stopwords padrão do spaCy para Português.
     - DOMAIN_TRASH ..... termos editoriais/HTML comuns ("ldquo","ndash", etc.).
     - ENTITIES_RE ...... regex para remover entidades HTML (&nbsp;).

  3) Função basic_clean(text, lower=True, strip_accents=False, keep_hyphen=True)
     - Remove tags HTML residuais e URLs.
     - Mantém hífens quando 'keep_hyphen=True' (p.ex. "primeiro-ministro"),para evitar quebrar compostos relevantes em PT.
     - Normaliza espaços e, se 'lower=True', passa a minúsculas.
     - Se 'strip_accents=True', usa unidecode para remover acentos (atenção:
       em PT pode alterar significado; por isso está a False por defeito).

  4) Função pre_clean(s)
     - Corrige Unicode com ftfy (arruma textos "partidos" por encoding).
     - Dupla decodificação HTML (html.unescape), depois remove entidades com ENTITIES_RE.
     - Aplica basic_clean (minúsculas, sem strip de acentos e mantendo hífen).
     - Resultado: string limpa, ainda NÃO tokenizada.

  5) Função spacy_tokenize_lemmatize(doc_text, remove_stop=True, only_alpha=True, min_len=3)
     - Tokeniza com spaCy e usa o LEMA (forma canónica) de cada token.
     - Filtros aplicados:
          * remove_stop=True  -> remove stopwords (ruído de função)
          * only_alpha=True   -> mantém só tokens alfabéticos (descarta nºs)
          * min_len=3         -> descarta tokens muito curtos (ruído)
     - Remove também itens do DOMAIN_TRASH.
     - Resultado: lista de lemas "limpos" prontos a análise.

**Porquê:**
  - A qualidade do texto de entrada determina a qualidade das features.
  - Lematizar reduz variação morfológica (p.ex. "doentes"/"doente" → "doente"),e melhora a frequência e peso nos modelos bag-of-words/TF-IDF.
  - Os filtros evitam que pontuação, palavras vazias ou "lixo editorial" dominem.

In [ ]:
import re, html, json
from collections import Counter
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from ftfy import fix_text
from unidecode import unidecode
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

# carregar modelo PT do spaCy (removemos o parser e o ner, para efeitos do exemplo)
nlp = spacy.load("pt_core_news_sm", disable=["parser", "ner"])

# stopwords do spaCy (PT); podem ser adicionadas mais
PT_STOP = nlp.Defaults.stop_words

# remoção de tokens “lixo” comuns de HTML/editoriais
DOMAIN_TRASH = {"ldquo","rdquo","laquo","raquo","ndash","mdash","hellip","apos","video","galeria","podcast"}

ENTITIES_RE = re.compile(r"&(#\d+|#x[0-9A-Fa-f]+|[A-Za-z0-9]+);")

def basic_clean(text: str,
                lower=True,
                strip_accents=False,
                keep_hyphen=True) -> str:
    """Limpeza canónica simples (sem quebrar PT)."""
    if not isinstance(text, str):
        text = "" if text is None else str(text)
    t = text

    # remover tags html que tenham escapado
    t = re.sub(r"<[^>]+>", " ", t)

    # opcional: remover urls
    t = re.sub(r"http\S+|www\.\S+", " ", t)

    # manter hífen se pedido (ex.: 'primeiro-ministro')
    if keep_hyphen:
        t = re.sub(r"[^0-9A-Za-zÀ-ÿ\- ]", " ", t)
    else:
        t = re.sub(r"[^0-9A-Za-zÀ-ÿ ]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    if lower:
        t = t.lower()
    if strip_accents:
        t = unidecode(t)
    return t

def pre_clean(s: str) -> str:
    """Corrigir unicode + decodificar entidades HTML + limpar."""
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    t = fix_text(s)
    t = html.unescape(t); t = html.unescape(t)  # forçar duas iterações
    t = ENTITIES_RE.sub(" ", t)
    t = basic_clean(t, lower=True, strip_accents=False, keep_hyphen=True)
    return t

def spacy_tokenize_lemmatize(doc_text: str,
                             remove_stop=True,
                             only_alpha=True,
                             min_len=3) -> list[str]:
    """Tokeniza com spaCy e devolve **lemas** filtrados."""
    doc = nlp(doc_text)
    out = []
    for tok in doc:
        lemma = tok.lemma_ if tok.lemma_ != "" else tok.text
        if remove_stop and lemma in PT_STOP:
            continue
        if only_alpha and not lemma.isalpha():
            continue
        if len(lemma) < min_len:
            continue
        out.append(lemma)
    # remove lixo editorial
    out = [w for w in out if w not in DOMAIN_TRASH]
    return out


**OBJETIVO:**
Criar um DataFrame simples com 2 documentos ("titulo", "texto") para demonstrar a pipeline. Mostra também como substituir por dados reais (CSV/JSON), que contenham as mesmas colunas. Ter o DataFrame de partida pronto para limpeza/tokenização.

**O QUE FAZ:**
  * Cria df = pd.DataFrame([...]) com colunas 'titulo' e 'texto'.

**Passos:**
  1) O pipeline assume a coluna 'texto' como fonte principal; 'titulo' é opcional e serve para rotular linhas (mais legível nos outputs).

**Notas**
  * Este bloco também mostra a "construção de top-k TF-IDF por documento" e tentativa de exportar CSV.
  * Se aparecer NameError: 'out_dir' não definido, define antes:
       from pathlib import Path
       out_dir = Path("sample_data"); out_dir.mkdir(exist_ok=True, parents=True)
e garante que a variável 'row_index' existe.

In [ ]:
# EXEMPLO mínimo com 2 textos:
data_example = [
    {"titulo": "Parlamento aprova medida",
     "texto": "O Parlamento deu luz verde à proposta. Nesse parlamento, Gabriel Bernardino falou à imprensa em Lisboa."},
    {"titulo": "ASF anuncia nova orientação",
     "texto": "A Autoridade de Supervisão de Seguros e Fundos de Pensões juntamente com o Parlamento publicou hoje novas regras."}
]
df = pd.DataFrame(data_example)

# PARA USAR OUTROS DATASETS:
#   a) Carregar CSV:
# df = pd.read_csv("/content/ficheiro.csv")  # garantir que tem as colunas: 'titulo' e 'texto'
#   b) Carregar JSON (lista de objetos):
# df = pd.read_json("/content/ficheiro.json")

# detetar colunas prováveis
prefer_text = ["texto","noticia","content","body","descricao","description"]
prefer_title = ["titulo","title","headline"]
cols = {c.lower(): c for c in df.columns}
text_col = next((cols[c] for c in prefer_text if c in cols), None)
title_col = next((cols[c] for c in prefer_title if c in cols), None)
if text_col is None:
    # fallback: primeira coluna de strings
    text_col = df.select_dtypes(include=["object"]).columns[0]
df = df[[c for c in [title_col, text_col] if c is not None]].dropna().drop_duplicates()
df = df.rename(columns={title_col or "": "titulo", text_col: "texto"})
print(df.head(2))
print(f"docs: {len(df)}")


**Objetivo:**
Aplicar as funções definidas atrás para obter:
* 'clean_text' ......... texto normalizado (string)
* 'tokens' ............. lista de lemas filtrados por doc
Obter uma representação por documento que alimenta visualizações e TF-IDF.


**Passos:**  
* df["clean_text"] = df["texto"].apply(pre_clean)
* df["tokens"] = df["clean_text"].apply(lambda t: spacy_tokenize_lemmatize(
    t, remove_stop=True, only_alpha=True, min_len=3))
* Imprime exemplo de tokens do primeiro documento (sanity check).

**Porquê:**
* Confirmar que as escolhas (stopwords, only_alpha, min_len) estão a produzir um vocabulário limpo e coerente para o teu domínio.


In [ ]:
# limpeza
df["clean_text"] = df["texto"].apply(pre_clean)

# tokenização + lematização
df["tokens"] = df["clean_text"].apply(lambda t: spacy_tokenize_lemmatize(
    t, remove_stop=True, only_alpha=True, min_len=3
))

# inspeção
print("Exemplo tokens do documento 0:", df["tokens"].iloc[0][:20])

**Objetivo:**
Análise exploratória do vocabulário resultante. Validar rapidamente a qualidade do vocabulário e ajustar filtros se preciso.

**Passos:**
* 'all_tokens' concatena tokens de todos os docs e calcula frequências (Counter).
* Imprime tamanho do vocabulário e top-20 termos.
* Desenha gráfico de barras com top-K termos (K = min(25, |voc|)).
* Gera uma wordcloud do corpus limpo.

**Porquê:**
* Perceber se o pré-processamento removeu ruído suficiente e se os termos com maior frequência fazem sentido no contexto (sanity check).

**Limitações:**
* Wordcloud e contagens “cruas” podem enviesar perceção (termos longos ou muito frequentes dominam visualmente). Para importância por documento, preferir TF-IDF.


In [ ]:
# Visualização global
all_tokens = [w for doc in df["tokens"] for w in doc]
freq = Counter(all_tokens)
print("Vocabulário:", len(freq))
print("Top 20 termos:", freq.most_common(20))

# gráfico de barras (top-K)
K = min(25, len(freq))
if K > 0:
    labels, values = zip(*freq.most_common(K))
    plt.figure(figsize=(12,4))
    plt.bar(labels, values)
    plt.xticks(rotation=45, ha="right")
    plt.title("Top termos (frequência)")
    plt.tight_layout()
    plt.show()

# wordcloud
if len(all_tokens) > 0:
    wc = WordCloud(width=1000, height=500, background_color="white")
    wc = wc.generate(" ".join(all_tokens))
    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title("Nuvem de palavras (corpus)")
    plt.show()


**Objetivo:** Converter cada documento numa representação numérica (vetor TF-IDF), permitindo medir importância relativa de termos por documento. Preparar terreno para ranking de palavras-chave, pesquisa por similaridade,clustering de documentos, etc

**Passos:**
* 'texts_joined' junta tokens por doc numa string (entrada do TfidfVectorizer).
* Cria TF-IDF com max_features=5000 (podes ajustar; ver notas abaixo).
* Calcula a DTM (Document-Term Matrix) esparsa X de shape (n_docs, n_terms).
* Define função top_tfidf_terms_for_doc(i, k) para retornar os k termos de maior peso TF-IDF no documento i.
* Imprime top-10 do documento 0 (exemplo).

**Porquê:**
* TF (Term Frequency) favorece o que é recorrente no doc.
* IDF (Inverse Document Frequency) penaliza termos demasiado comuns no corpus.
* O produto destaca termos “característicos” daquele documento.


**Notas:**
PARÂMETROS ÚTEIS A AFINAR (quando aplicares a corpora maiores):
* ngram_range=(1,2) .......... ativa bigramas (pode capturar expressões estáveis)
* min_df / max_df ............ remove termos raros/dominantes (ex.: min_df=3, max_df=0.8)
* max_features ............... limita vocabulário (eficiência/overfitting)

In [ ]:
# juntar os tokens por doc (string) para alimentar o TF-IDF
texts_joined = df["tokens"].apply(lambda xs: " ".join(xs)).tolist()

# TF-IDF clássico (unigramas). Ajusta max_features se for necessário limitar.
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(texts_joined)  # shape: (n_docs, n_terms)
terms = tfidf.get_feature_names_out()
print("DTM TF-IDF:", X.shape)

# Top termos por documento (k melhores)
def top_tfidf_terms_for_doc(row_index, k=10):
    row = X.getrow(row_index)
    if row.nnz == 0:
        return []
    # termos ordenados por peso decrescente
    inds = row.indices
    vals = row.data
    order = vals.argsort()[::-1][:k]
    return [(terms[inds[i]], float(vals[i])) for i in order]

# exemplo: top-10 do 1º documento
print("Top-10 TF-IDF (documento 0):", top_tfidf_terms_for_doc(0, k=10))


**Objetivo:**
Produzir um resumo estruturado dos termos mais relevantes (por TF-IDF) para todos os documentos e guardar em CSV. Deixar em disco um artefacto reutilizável (CSV) com os “top termos por doc”, pronto para integrar noutros sistemas/processos.

**Passos:**
* out_dir = Path("sample_data") ................ define pasta de saída. (cria manualmente com out_dir.mkdir(exist_ok=True, parents=True) se necessário)
* Reajusta o TF-IDF sobre 'texts_joined' (mesma configuração da célula 7).
* Define 'row_index' (rótulos das linhas):
* Se existir 'titulo' completo, usa-o (truncado a 80 chars para legibilidade); caso contrário, usa "doc_i" por índice.
* Para cada documento:
    * extrai os índices e pesos dos termos não nulos;
    * ordena por peso decrescente;
    * recolhe os top-k (ex.: k=10) com: doc_idx, doc_label, rank, term, tfidf;
* Concatena tudo num DataFrame 'df_topk', mostra um head(20) para inspeção.
* Escreve CSV: sample_data/tfidf_top{k}_per_doc.csv

**Porquê:**
* Gera um “resumo automático” do conteúdo de cada documento sem supervisionar.
* Útil para: etiquetas rápidas, pesquisa, triagem manual, criação de dicionários.


**Erros Comuns:**
* Se 'out_dir' NÃO existir, a escrita do CSV falha → cria a pasta antes.
* Se 'row_index' usar 'titulo' com NaN, converte para string segura: row_index = [str(t) if pd.notna(t) else f"doc_{i}" for i,t in enumerate(df["titulo"])]
* Vocab pequeno + docs curtos podem dar linhas sem termos (row.nnz == 0).

In [ ]:
# =========================
# MATRIZ TF-IDF COMPLETA
# =========================
from scipy import sparse

out_dir = Path("sample_data")

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(texts_joined)   # shape: (n_docs, n_terms)
terms = tfidf.get_feature_names_out()

print("DTM TF-IDF:", X.shape)

# índice de linhas (usa títulos se existirem, senão usa doc_i)
if "titulo" in df.columns and df["titulo"].notna().all():
    row_index = [str(t)[:80] for t in df["titulo"].tolist()]  # corta para ficar legível
else:
    row_index = [f"doc_{i}" for i in range(len(df))]

# Converter para denso só se a matriz for pequena
max_cells_to_dense = 2000000  # ~2M células; ajustar conforme a RAM
n_cells = X.shape[0] * X.shape[1]

if n_cells <= max_cells_to_dense:
    import numpy as np
    df_tfidf = pd.DataFrame(X.toarray(), columns=terms, index=row_index)
    display(df_tfidf.round(3).head())         # preview
    # guardar CSV da matriz densa
    out_csv = out_dir / "tfidf_matrix_dense.csv"
    df_tfidf.to_csv(out_csv, index=True, encoding="utf-8")
    print("Matriz TF-IDF densa guardada em:", out_csv)
else:
    print("Matriz grande — a guardar em formato esparso (.npz) + termos...")
    # guardar matriz esparsa + termos (forma recomendada para grandes)
    out_npz = out_dir / "tfidf_matrix_sparse.npz"
    sparse.save_npz(out_npz, X)
    (out_dir / "tfidf_terms.txt").write_text("\n".join(terms), encoding="utf-8")
    (out_dir / "tfidf_rows.txt").write_text("\n".join(row_index), encoding="utf-8")
    print("Guardado:", out_npz, "(e tfidf_terms.txt, tfidf_rows.txt)")

# =========================
# TOP-k TERMOS POR DOCUMENTO (sem densificar)
# =========================
def top_tfidf_terms_for_doc(row_index_int: int, k: int = 10):
    row = X.getrow(row_index_int)
    if row.nnz == 0:
        return []
    inds = row.indices
    vals = row.data
    order = vals.argsort()[::-1][:k]
    return [(terms[inds[i]], float(vals[i])) for i in order]

# construir tabela com top-k para TODOS os docs
k = 10
rows = []
for i in range(X.shape[0]):
    tops = top_tfidf_terms_for_doc(i, k=k)
    for rank, (term, score) in enumerate(tops, start=1):
        rows.append({
            "doc_idx": i,
            "doc_label": row_index[i],
            "rank": rank,
            "term": term,
            "tfidf": score
        })

df_topk = pd.DataFrame(rows)
display(df_topk.head(20))

# guardar top-k em CSV
out_topk = out_dir / f"tfidf_top{str(k)}_per_doc.csv"
df_topk.to_csv(out_topk, index=False, encoding="utf-8")
print("Top-k por documento guardado em:", out_topk)